In [4]:
import cvxpy as cp
import numpy as np

In [76]:
# Definir variáveis k, f
k, f = 1, 1
# Definir as matrizes A, B1, B2, C e D
A = np.array([[0, 0, 1, 0],
              [0, 0, 0, 1],
              [-k, k, -f, f],
              [k, -k, f, -f]])

B1 = np.array([[0], [0], [0], [1]])

B2 = np.array([[0], [0], [0], [1]])

C = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 0]])

D = np.array([[0], [0], [1]])

n1, n2, n3 = A.shape[0], C.shape[0], B1.shape[1]

# Definir as variáveis de otimização
W = cp.Variable((n1, n1), symmetric=True)
Z = cp.Variable((1, n1))
mu = cp.Variable()

# Blocos para LMI
M11 = A@W + W@A.T + B2@Z + Z.T@B2.T
M12 = W@C.T + Z.T@D.T
M13 = B1
M21 = C@W + D@Z
M22 = -np.eye(n2)
M23 = np.zeros((n2,n3))
M31 = B1.T
M32 = np.zeros((n3, n2))
M33 = -mu * np.eye(n3)

# Construir a matriz de blocos LMI
LMI = cp.bmat([
    [M11, M12, M13],
    [M21, M22, M23],
    [M31, M32, M33]
])

# Definir a função objetivo: min (mu)
objective = cp.Minimize(mu)

# Definir as restrições
constraints = [LMI << 0]

# Criar e resolver o problema de otimização
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCS) # É uma boa prática especificar o solver

# Verificar o status da solução
if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print("Status:", problem.status, "\n")
    print("Valor ótimo de mu:",problem.value)
    print("Matriz W:\n",W.value)
    print("Vetor Z:\n",Z.value)
else:
    print("O problema não tem uma solução ótima. Status:", problem.status)

Status: optimal 

Valor ótimo de mu: 0.9999850552108828
Matriz W:
 [[ 0.4548729  -0.46382445 -0.35856482  0.35852127]
 [-0.46382445  0.4742304   0.36379101 -0.36390334]
 [-0.35856482  0.36379101  1.81102143 -1.81128694]
 [ 0.35852127 -0.36390334 -1.81128694  1.81158138]]
Vetor Z:
 [[-0.54288259  0.53627228  2.35319474 -3.35346087]]


In [77]:
Z.value @ np.linalg.inv(W.value)

array([[ -13246.7974753 ,  -12953.14276858, -105151.82060544,
        -105116.94802385]])